In [39]:
#mport numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D,BatchNormalization 
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
import pandas as pd

In [40]:
data = pd.read_csv("train.csv")
data_test=pd.read_csv("test.csv")
data.shape


(42000, 785)

In [41]:
# processing of data
Y_train = data["label"]
X_train = data.drop(labels=["label"],axis=1)

In [42]:
X_train = X_train/255.0
X_train = X_train.astype('float32')
X_test = data_test/255.0
X_test = X_test.astype('float32')
X_train = X_train.values.reshape(X_train.shape[0],28,28,1)
X_test = X_test.values.reshape(X_test.shape[0],28,28,1)

Y_train = to_categorical(Y_train,num_classes=10)

In [44]:
datagen =ImageDataGenerator(
         rotation_range=10,
         zoom_range=0.1,
         width_shift_range=0.1,
         height_shift_range=0.1 )
datagen.fit(X_train)
X_train,X_val,Y_train,Y_val = train_test_split(X_train,Y_train,test_size=0.2)

In [50]:
model=Sequential()

model.add(Conv2D(32,kernel_size=5,input_shape=(28,28,1),activation='relu'))
model.add(Conv2D(32,kernel_size=5,activation='relu'))
model.add(MaxPool2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(64,kernel_size=3,activation='relu'))
model.add(Conv2D(64,kernel_size=3,activation='relu'))
model.add(MaxPool2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(128,kernel_size=3,activation='relu'))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.4))
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.4))
model.add(Dense(10, activation = "softmax"))


In [51]:
optimizer=Adam(lr=0.01)
model.compile(optimizer=optimizer,loss = "categorical_crossentropy", metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 20, 20, 32)        25632     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 10, 32)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 10, 10, 32)        128       
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 10, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 8, 8, 64)          18496     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 6, 6, 64)          36928     
__________

In [53]:
model_try = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=64),
                              epochs = 30, validation_data = (X_val,Y_val),
                              verbose = 1, steps_per_epoch=300)

predictions = model.predict(X_test)
predictions = np.argmax(predictions,axis = 1)
predictions = pd.Series(predictions, name="Label")
submit = pd.concat([pd.Series(range(1,28001),name = "ImageId"),predictions],axis = 1)
submit.to_csv("result.csv",index=False)

Epoch 1/30
300/300 [==============================] - 162s 539ms/step - loss: 0.2104 - acc: 0.9565 - val_loss: 0.0655 - val_acc: 0.9861
Epoch 2/30
300/300 [==============================] - 169s 562ms/step - loss: 0.1676 - acc: 0.9646 - val_loss: 0.0587 - val_acc: 0.9875
Epoch 3/30
300/300 [==============================] - 174s 581ms/step - loss: 0.1603 - acc: 0.9667 - val_loss: 0.1574 - val_acc: 0.9694
Epoch 4/30
300/300 [==============================] - 191s 637ms/step - loss: 0.1677 - acc: 0.9636 - val_loss: 0.0716 - val_acc: 0.9882
Epoch 5/30
300/300 [==============================] - 198s 660ms/step - loss: 0.1652 - acc: 0.9654 - val_loss: 0.0581 - val_acc: 0.9875
Epoch 6/30
300/300 [==============================] - 163s 543ms/step - loss: 0.1823 - acc: 0.9629 - val_loss: 0.0759 - val_acc: 0.9862
Epoch 7/30
300/300 [==============================] - 154s 513ms/step - loss: 0.1708 - acc: 0.9640 - val_loss: 0.0602 - val_acc: 0.9881
Epoch 8/30
300/300 [============================